In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
from dollarpy import Recognizer, Template, Point
# import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from collections import defaultdict
from queue import Empty
import socket
import pickle 
import json
import time
import base64

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [2]:
def poseFunction(frame):
     with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        results = holistic.process(image)
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                    )

        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                    )

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                    )
        
        print(results.pose_landmarks)
        return image , results.pose_landmarks.landmark


In [ ]:
def CreatePoints(videoURL,label):
    cap = cv2.VideoCapture(videoURL)
    L_All_point = {}
    L_selected_points = [11,12,13,14,15,16,17,18,19,20,23,24]
    for s in L_selected_points:
        L_All_point[s] = []

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        #List to hold Coordinates
        points = []
        f = {}
    
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == False:
                break
            if ret==True:
              
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        

                results = holistic.process(image)

                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                         )


                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                         )


                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                         )

                try:

                    pose = results.pose_landmarks.landmark
                    index = 0
                    newlist=[]
                    for s in L_All_point:
                        p_s = Point(results.pose_landmarks.landmark[s].x , results.pose_landmarks.landmark[s].y , 1)
                        L_All_point[s].append(p_s)

                except:
                    pass
                        
                
             
             
                #cv2.imshow(label, image)

            

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    t = []
    for f in L_All_point:
        t += L_All_point[f]
    points = t
    with open('ExercisesData.csv', 'a+' , newline = '') as f:
        writer = csv.writer(f)
        L = []
        
        class_name = label
        L.append(class_name)

        for f in L_All_point:
            for j in L_All_point[f]:
                pt_x = round(j.x,4)
                pt_y = round(j.y,4)
                point_sep = str(pt_x) + "," + str(pt_y)
                L.append(point_sep)
            L.append(',')
        

        writer.writerow(L)
        
    return points

In [ ]:
#vid = "stomach_hip_thrust.mp4"

#points = CreatePoints(vid,"stomach,hip_thrust")
 
#tmpl_2 = Template('hook', points)



In [3]:
def GetDataFromCSV():
    import pandas as pd
    import csv
    from collections import defaultdict
    from queue import Empty
    import csv
    
    my_dict = defaultdict(list)
    with open('ExercisesData.csv', newline='') as f:
        reader = csv.reader(f)
        data = list(reader)

    data.pop(0)
    output = []
    temp = []
    outer = defaultdict(list)
    class_name = ""
    for h in data:
        flag = 0
        output = []
        for item in h:
            if flag == 0:
                flag = 1
                class_name = item
                continue
            if item == ',':
                output.append(temp)
                temp = []
                continue
            st = item.split(',')
            pt_x = float(st[0])
            pt_y = float(st[1])
            f = Point(pt_x , pt_y , 1)
            temp.append(f)
    
        outer[class_name].append(output)
    
    for o in outer:
        print("before")
        print(outer[o][0])
        outer[o][0] = sum(outer[o][0] , [])
        print("after")
        print(outer[o][0])

    temp_list = []
    for t in outer:
        tmp_curr = Template(t,outer[t][0])       
        temp_list.append(tmp_curr)  
    return temp_list


In [4]:

Templates = GetDataFromCSV()
print(Templates[3].name)


before
[[(0.6853, 0.604), stroke 1, (0.6855, 0.5998), stroke 1, (0.6855, 0.5974), stroke 1, (0.6856, 0.5954), stroke 1, (0.6856, 0.5942), stroke 1, (0.6857, 0.593), stroke 1, (0.6857, 0.5927), stroke 1, (0.6859, 0.5928), stroke 1, (0.686, 0.5929), stroke 1, (0.6863, 0.593), stroke 1, (0.6864, 0.593), stroke 1, (0.6864, 0.5931), stroke 1, (0.6864, 0.5932), stroke 1, (0.6864, 0.5933), stroke 1, (0.6864, 0.5933), stroke 1, (0.6864, 0.5933), stroke 1, (0.6864, 0.5933), stroke 1, (0.6864, 0.5934), stroke 1, (0.6864, 0.5934), stroke 1, (0.6864, 0.5935), stroke 1, (0.6865, 0.5936), stroke 1, (0.6865, 0.5937), stroke 1, (0.6866, 0.5941), stroke 1, (0.6866, 0.5944), stroke 1, (0.6867, 0.5948), stroke 1, (0.6867, 0.5951), stroke 1, (0.6868, 0.5954), stroke 1, (0.6869, 0.5957), stroke 1, (0.6869, 0.5959), stroke 1, (0.687, 0.596), stroke 1, (0.687, 0.5962), stroke 1, (0.6871, 0.5963), stroke 1, (0.6872, 0.5966), stroke 1, (0.6873, 0.597), stroke 1, (0.6874, 0.5971), stroke 1, (0.6874, 0.5972), st

In [8]:


#testing

host = '127.0.0.1'
port = 10000

    # Create a socket object
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((host, port))
server_socket.listen()
conn , addr = server_socket.accept()
print("device connected")
id_marker = {"1": "Arms", "2":"Stomach" , "3":"Thighs" , "4":"Legs"}

cap = cv2.VideoCapture('C:/Users/NoorLaptop/Desktop/HCI_Phase_1/Legs_Extension.mp4')
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('http://192.168.8.101:4747/video')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
out = cv2.VideoWriter('Experiment.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, size)
L2_point = []
L_selected_points = [11,12,13,14,15,16,17,18,19,20,23,24]
L_test_point = []
m = {}
#recognizer = Recognizer([Templates[0]])
recognizer = Recognizer(Templates)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
   ct = 0
   while(True):
        ct += 1
        ret, frame = cap.read()
    
        if frame is None:
            break
    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        results = holistic.process(image)
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        for f in L_selected_points:
            if(results.pose_landmarks is not None):
                p = Point(results.pose_landmarks.landmark[f].x, results.pose_landmarks.landmark[f].y , 1)
                L_test_point.append(p)  


        '''
        #message = data.decode('utf-8')
        #print(message)
        if(ct % 5 == 0):
         data = conn.recv(1024)
         print(ct)
         '''
        with open('C:/Users/NoorLaptop/Desktop/HCI_Phase_1/TUIO_Latest/TUIO11_NET-master/bin/Debug/ID_passer.txt') as f:
          contents = f.read()
        
        if(ct % 15 == 0):
            recognizer = Recognizer(Templates)
            result = recognizer.recognize(L_test_point)
            res_str = result[0]
            perc_str = str(round(result[1],3))
            final_res = ""
            true_val = id_marker[contents].lower()
            print(true_val)
            if(res_str is not None):
             v = res_str.split(',')
             k = v[0].lower()
             fg = k == true_val
             final_res = res_str + perc_str + str(fg) + '\n'
            
           
            conn.send(final_res.encode('utf-8'))
            if(res_str is not None):   
                res_info = 'the label is ' + res_str
                res_pr = 'with ' + perc_str 
                font = cv2.FONT_HERSHEY_SIMPLEX
                org = (50, 50)
                fontScale = 1
                color = (255, 0, 0)
                thickness = 1
                cv2.putText(frame, res_info, org, font, 
                        fontScale, color, thickness, cv2.LINE_AA)  
                org2 = (50 , 100)
                cv2.putText(frame, res_pr, org2, font, 
                        fontScale, color, thickness, cv2.LINE_AA)  
                
            L_test_point = []


        cv2.imshow('Show',frame)       
        #out.write(image)
     
   
        
         
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

conn.close()
cap.release()
cv2.destroyAllWindows()
 


device connected
legs
legs
legs
legs
legs
legs
legs


In [8]:
from twilio.rest import Client
account_sid = 'ACf3af298941b9c8918711b0e060ece907'
auth_token = 'e1e30962f7ed51792e227cf8877739bc'
twilio_number = '+15076186720'
my_phone_number = '+201068449753'
with open('C:/Users/NoorLaptop/Desktop/HCI_Phase_1/TUIO_Latest/TUIO11_NET-master/bin/Debug/Messaging.txt') as f:
    cont = f.read()
client = Client(account_sid , auth_token)
message = client.messages.create(body = cont , from_ = twilio_number, to=my_phone_number)

In [ ]:

host = '127.0.0.1'
port = 10000

    # Create a socket object
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((host, port))
server_socket.listen()
conn , addr = server_socket.accept()
print("device connected")
cap = cv2.VideoCapture('C:/Users/NoorLaptop/Desktop/HCI_Phase_1/walking.mp4')
#cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
out = cv2.VideoWriter('Experiment.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, size)
L2_point = []
L_selected_points = [11,12,13,14,15,16,17,18,19,20,23,24]
L_test_point = []
m = {}
#recognizer = Recognizer([Templates[0]])
recognizer = Recognizer(Templates)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
   ct = 0
   while(True):
        ct += 1
        ret, frame = cap.read()
        cv2.imshow('Show',frame)
        '''
        if frame is None:
            break
        '''
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        results = holistic.process(image)
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        for f in L_selected_points:
            p = Point(results.pose_landmarks.landmark[f].x, results.pose_landmarks.landmark[f].y , 1)
            L_test_point.append(p)  


        '''
        #message = data.decode('utf-8')
        #print(message)
        if(ct % 5 == 0):
         data = conn.recv(1024)
         print(ct)
         '''
        with open('ID_passer.txt') as f:
          contents = f.read()
          print(contents)
        
        if(ct % 2 == 0):
            recognizer = Recognizer(Templates)
            result = recognizer.recognize(L_test_point)
            res_str = result[0]
            perc_str = str(result[1])
            final_res = ""
            if(res_str is not None):
             final_res = res_str + '\n'
            print(res_str)
            conn.send(final_res.encode('utf-8'))
            if(res_str is not None):   
                res_info = 'the label is ' + res_str
                res_pr = 'with ' + perc_str 
                font = cv2.FONT_HERSHEY_SIMPLEX
                org = (50, 50)
                fontScale = 1
                color = (255, 0, 0)
                thickness = 1
                cv2.putText(image, res_info, org, font, 
                        fontScale, color, thickness, cv2.LINE_AA)  
                org2 = (50 , 100)
                cv2.putText(image, res_pr, org2, font, 
                        fontScale, color, thickness, cv2.LINE_AA)  
            L_test_point = []

       
        #out.write(image)
     
   
        
         
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

conn.close()
cap.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture('http://192.168.8.101:4747/video')

while(True):
    ret, frame = cap.read()

    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()